In [1]:
#Create directory Structure
import os
os.makedirs('project/agents', exist_ok=True)
os.makedirs('project/tools', exist_ok=True)
os.makedirs('project/memory', exist_ok=True)
os.makedirs('project/core', exist_ok=True)

```mermaid
graph TD
    subgraph Multi-Agent Workflow
        A[User Input] --> 1. handle_message(user_input) --> B(Main Agent)
        B -- 2. Log Input --> C[Observability/Logger]

        B -- 3. Create Context --> D[Context Engineering]
        D -- 3.1 Access Memory --> E[Session Memory]
        D -- Context Data --> B

        B -- 4. Plan Task --> F[Planner]
        F -- 4.1 Access Memory --> E
        F -- Plan Data --> B

        B -- 5. Execute Plan --> G[Worker]
        G -- Work Output Data --> B

        B -- 6. Evaluate Output --> H[Evaluator]
        H -- Evaluation Result --> B

        B -- 7. Return Response --> I[Final Response]

        %% Conceptual A2A Trace Links (Main Agent acts as orchestrator)
        B --- Data Flow -- F
        B --- Data Flow -- G
        B --- Data Flow -- H

        F -- A2A Trace --> J[A2A Protocol]
        G -- A2A Trace --> J
        H -- A2A Trace --> J
    end

    style A fill:#f9f,stroke:#333,stroke-width:2px
    style B fill:#bbf,stroke:#333,stroke-width:2px
    style C fill:#ccf,stroke:#333,stroke-width:2px
    style D fill:#ddf,stroke:#333,stroke-width:2px
    style E fill:#eef,stroke:#333,stroke-width:2px
    style F fill:#dfd,stroke:#333,stroke-width:2px
    style G fill:#fdd,stroke:#333,stroke-width:2px
    style H fill:#ffd,stroke:#333,stroke-width:2px
    style I fill:#cfc,stroke:#333,stroke-width:2px
    style J fill:#fcf,stroke:#333,stroke-width:2px
```

In [2]:
%%writefile project/agents/planner.py
class Planner:
    def plan(self, user_input, memory):
       # Simple parsing logic (can be replaced with NLP later)
        user_input_lower = user_input.lower()

        start, dest = None, None
        mobility = memory.load("mobility") or "any"

        # Extract start/destination heuristically
        if "from" in user_input_lower and "to" in user_input_lower:
            parts = user_input_lower.split("to")
            start = parts[0].split("from")[-1].strip()
            dest = parts[1].strip()
        else:
            start = "current location"
            dest = "nearest relief point"

        # Detect hazards
        hazards = []
        if "flood" in user_input_lower:
            hazards.append("flood")
        if "earthquake" in user_input_lower:
            hazards.append("earthquake")
        if "aftershock" in user_input_lower:
            hazards.append("aftershock")
        if "road blocked" in user_input_lower or "closed" in user_input_lower:
            hazards.append("road_block")

        plan = {
            "start": start,
            "destination": dest,
            "mobility": mobility,
            "hazards": hazards
        }

        return plan





Writing project/agents/planner.py


In [3]:
%%writefile project/agents/worker.py
class Worker:
    def execute(self, plan):
        start = plan.get("start", "current location")
        dest = plan.get("destination", "nearest relief point")
        mobility = plan.get("mobility", "any")
        hazards = plan.get("hazards", [])

        # Simple static map dataset
        map_routes = {
            ("lakeview colony", "emergency relief camp"): ["Oak Street", "Hill Lane", "East Service Road"],
            ("green market", "nearest open shelter"): ["West Bypass Road", "Industrial Circle", "Shelter Point A"],
            ("hill road", "school relief center"): ["Pine Street", "North Lane", "School Relief Center"],
            ("railway station", "city hospital"): ["Ground Road", "Market Street", "City Hospital Road"],
            ("old town", "medical triage point"): ["Broad Avenue", "Field Road", "Triage Point"]
        }

        # Default route if not found
        key = (start.lower(), dest.lower())
        route = map_routes.get(key, ["Route step 1", "Route step 2", "Route step 3"])

        # Adjust for hazards
        if "flood" in hazards:
            route = [r + " (avoid waterlogged areas)" for r in route]
        if "earthquake" in hazards or "aftershock" in hazards:
            route = [r + " (prefer open areas)" for r in route]

        # Adjust for mobility preference
        if mobility == "walking":
            route = [r + " (walking-friendly)" for r in route]

        return {
            "route": " → ".join(route),
            "raw_output": route
        }


Writing project/agents/worker.py


In [4]:
%%writefile project/agents/evaluator.py
class Evaluator:
    def evaluate(self, worker_output):
        route_text = worker_output.get("route", "Route unavailable")
        status = "approved" if route_text else "needs review"
        return {
            "response": route_text,
            "status": status
        }

Writing project/agents/evaluator.py


In [5]:
%%writefile project/tools/tools.py
class Tools:
    def calculate(self,x,y): return x+y
    def summarize(self,text): return text[:100]

Writing project/tools/tools.py


In [6]:
%%writefile project/memory/session_memory.py
class SessionMemory:
    def __init__(self): self.data={}
    def save(self,k,v): self.data[k]=v
    def load(self,k): return self.data.get(k)

Writing project/memory/session_memory.py


In [7]:
%%writefile project/core/context_engineering.py
class ContextEngineering:
    def create_context(self,user_input,memory): return f"Context: {user_input}"

Writing project/core/context_engineering.py


In [8]:
%%writefile project/core/observability.py
class Observability:
    def log(self,msg): print(f"[LOG] {msg}")

Writing project/core/observability.py


In [9]:
%%writefile project/core/a2a_protocol.py
class A2AProtocol:
    def pass_message(self,sender,receiver,message): return receiver(message)

Writing project/core/a2a_protocol.py


In [10]:
%%writefile project/main_agent.py
from project.agents.planner import Planner
from project.agents.worker import Worker
from project.agents.evaluator import Evaluator
from project.memory.session_memory import SessionMemory
from project.core.context_engineering import ContextEngineering
from project.core.observability import Observability
from project.core.a2a_protocol import A2AProtocol

class MainAgent:
    def __init__(self):
        self.planner=Planner(); self.worker=Worker(); self.evaluator=Evaluator()
        self.memory=SessionMemory(); self.context_engineering=ContextEngineering()
        self.logger=Observability(); self.protocol=A2AProtocol()

    def handle_message(self,user_input):
        self.logger.log("Received input")
        context=self.context_engineering.create_context(user_input,self.memory)
        plan=self.planner.plan(context,self.memory)
        work=self.worker.execute(plan)
        return self.evaluator.evaluate(work)

def run_agent(user_input:str):
    agent=MainAgent(); result=agent.handle_message(user_input); return result["response"]

Writing project/main_agent.py


In [11]:
%%writefile project/app.py
from project.main_agent import run_agent
if __name__=='__main__': print(run_agent("Hello from app!"))

Writing project/app.py


In [12]:
%%writefile project/run_demo.py
import sys,os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__),"..")))
from project.main_agent import run_agent
if __name__=='__main__': print(run_agent("Hello! This is a demo."))

Writing project/run_demo.py


In [13]:
%%writefile project/requirements.txt
python-dotenv
requests
tqdm
numpy
pandas

Writing project/requirements.txt


In [14]:
from project.main_agent import run_agent
print(run_agent("There's heavy flooding near the central bridge. I need a safe walking route from Lakeview Colony to the Emergency Relief Camp."))

[LOG] Received input
Route step 1 (avoid waterlogged areas) → Route step 2 (avoid waterlogged areas) → Route step 3 (avoid waterlogged areas)


```mermaid
graph TD
    subgraph Multi-Agent Workflow
        A[User Input] --> 1. handle_message(user_input) --> B(Main Agent)
        B -- 2. Log Input --> C[Observability/Logger]

        B -- 3. Create Context --> D[Context Engineering]
        D -- 3.1 Access Memory --> E[Session Memory]
        D -- Context Data --> B

        B -- 4. Plan Task --> F[Planner]
        F -- 4.1 Access Memory --> E
        F -- Plan Data --> B

        B -- 5. Execute Plan --> G[Worker]
        G -- Work Output Data --> B

        B -- 6. Evaluate Output --> H[Evaluator]
        H -- Evaluation Result --> B

        B -- 7. Return Response --> I[Final Response]

        %% Conceptual A2A Trace Links (Main Agent acts as orchestrator)
        B --- Data Flow -- F
        B --- Data Flow -- G
        B --- Data Flow -- H

        F -- A2A Trace --> J[A2A Protocol]
        G -- A2A Trace --> J
        H -- A2A Trace --> J
    end

    style A fill:#f9f,stroke:#333,stroke-width:2px
    style B fill:#bbf,stroke:#333,stroke-width:2px
    style C fill:#ccf,stroke:#333,stroke-width:2px
    style D fill:#ddf,stroke:#333,stroke-width:2px
    style E fill:#eef,stroke:#333,stroke-width:2px
    style F fill:#dfd,stroke:#333,stroke-width:2px
    style G fill:#fdd,stroke:#333,stroke-width:2px
    style H fill:#ffd,stroke:#333,stroke-width:2px
    style I fill:#cfc,stroke:#333,stroke-width:2px
    style J fill:#fcf,stroke:#333,stroke-width:2px
```

In [15]:
!zip -r project.zip project

  adding: project/ (stored 0%)
  adding: project/tools/ (stored 0%)
  adding: project/tools/tools.py (deflated 22%)
  adding: project/run_demo.py (deflated 31%)
  adding: project/app.py (deflated 16%)
  adding: project/core/ (stored 0%)
  adding: project/core/a2a_protocol.py (deflated 21%)
  adding: project/core/observability.py (deflated 2%)
  adding: project/core/__pycache__/ (stored 0%)
  adding: project/core/__pycache__/a2a_protocol.cpython-312.pyc (deflated 34%)
  adding: project/core/__pycache__/observability.cpython-312.pyc (deflated 33%)
  adding: project/core/__pycache__/context_engineering.cpython-312.pyc (deflated 35%)
  adding: project/core/context_engineering.py (deflated 16%)
  adding: project/requirements.txt (stored 0%)
  adding: project/agents/ (stored 0%)
  adding: project/agents/planner.py (deflated 64%)
  adding: project/agents/worker.py (deflated 57%)
  adding: project/agents/__pycache__/ (stored 0%)
  adding: project/agents/__pycache__/evaluator.cpython-312.pyc (d